In [10]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
import warnings
warnings.filterwarnings('ignore')
import time
# --- Library Imports ---
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import optuna
print("Libraries imported successfully.")
# --- Helper Function for Winkler Score ---
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)
# --- Global Constants ---
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
N_OPTUNA_TRIALS = 30 # A strong number for a comprehensive search
COMPETITION_ALPHA = 0.1

# --- Load Raw Data ---
try:
    # We drop the low-variance columns they identified right away
    drop_cols=['id', 'golf', 'view_rainier', 'view_skyline', 'view_lakesamm','view_otherwater', 'view_other']
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv').drop(columns=drop_cols)
    df_test = pd.read_csv(DATA_PATH + 'test.csv').drop(columns=drop_cols)
    print("Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()
# --- Prepare Target Variable ---
y_true = df_train['sale_price'].copy()
# The mean-error model works best when predicting the raw price directly
# So, we will NOT log-transform the target this time.
# df_train.drop('sale_price', axis=1, inplace=True) # We keep sale_price for FE
print("Setup complete.")


Libraries imported successfully.
Raw data loaded successfully.
Setup complete.


In [11]:
# =============================================================================
# BLOCK 2: SYNTHESIZED FEATURE ENGINEERING (CORRECTED)
# =============================================================================
print("--- Starting Block 2: Synthesized Feature Engineering ---")
def create_synthesized_features(df_train, df_test):
    # Combine for consistent processing and reset the index
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    # Store the original id for later, as reset_index will remove it
    train_ids = df_train.index
    test_ids = df_test.index
    all_data = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)
    
    # --- A) Brute-Force Numerical Interactions ---
    print("Creating brute-force numerical interaction features...")
    NUMS = ['area', 'land_val', 'imp_val', 'sqft_lot', 'sqft', 'sqft_1','grade', 'year_built']
    for i in range(len(NUMS)):
        for j in range(i + 1, len(NUMS)):
            all_data[f'{NUMS[i]}_x_{NUMS[j]}'] = all_data[NUMS[i]] *all_data[NUMS[j]]
    
    # --- B) Date Features ---
    all_data['sale_date'] = pd.to_datetime(all_data['sale_date'])
    all_data['year'] = all_data['sale_date'].dt.year
    all_data['month'] = all_data['sale_date'].dt.month
    all_data['year_diff'] = all_data['year'] - all_data['year_built']
    
    # --- C) TF-IDF Text Features ---
    print("Creating TF-IDF features for text columns...")
    text_cols = ['subdivision', 'zoning', 'city', 'sale_warning','join_status', 'submarket']
    all_data[text_cols] = all_data[text_cols].fillna('missing').astype(str)
    for col in text_cols:
        tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5),max_features=128, binary=True)
        tfidf_matrix = tfidf.fit_transform(all_data[col])
        svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
        tfidf_svd = svd.fit_transform(tfidf_matrix)
        tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
        
        # This concat will now work because both have a simple 0-based index
        all_data = pd.concat([all_data, tfidf_df], axis=1)
    
    # --- D) Log transform some of the new interaction features ---
    for c in ['land_val_x_imp_val', 'land_val_x_sqft', 'imp_val_x_sqft']:
        if c in all_data.columns:
            # Add a small constant to avoid log(0)
            all_data[c] = np.log1p(all_data[c].fillna(0))
    
    # --- E) Final Cleanup ---
    print("Finalizing feature set...")
    cols_to_drop = ['sale_date', 'subdivision', 'zoning', 'city','sale_warning', 'join_status', 'submarket']
    all_data = all_data.drop(columns=cols_to_drop)
    all_data.fillna(0, inplace=True)
    
    # Separate final datasets
    X = all_data[all_data['is_train'] == 1].drop(columns=['is_train','sale_price'])
    X_test = all_data[all_data['is_train'] == 0].drop(columns=['is_train','sale_price'])
    
    # Restore the original 'id' as the index
    X.index = train_ids
    X_test.index = test_ids
    X_test = X_test[X.columns]
    return X, X_test

# We need to re-run this from the original dataframes
X, X_test = create_synthesized_features(df_train, df_test)
print(f"\nSynthesized FE complete. Total features: {X.shape[1]}")
gc.collect()


--- Starting Block 2: Synthesized Feature Engineering ---
Creating brute-force numerical interaction features...
Creating TF-IDF features for text columns...
Finalizing feature set...

Synthesized FE complete. Total features: 111


16

In [12]:
# =============================================================================
# BLOCK 3: K-FOLD AWARE TUNING & TRAINING OF MEAN MODEL
# =============================================================================
print("\n--- STAGE 1, PART 1: K-Fold Aware Tuning of Mean Model ---")

# Reload 'grade' for stratification as it's a reliable column
grade_for_stratify = pd.read_csv(DATA_PATH + 'dataset.csv')['grade']

def objective_mean_kfold(trial):
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method':'hist',
        'random_state': RANDOM_STATE, 'n_jobs': -1,
        'eta': trial.suggest_float('eta', 0.035, 0.045),
        'max_depth': trial.suggest_int('max_depth', 7, 9),
        'subsample': trial.suggest_float('subsample', 0.80, 0.88),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.80, 0.88),
        'lambda': trial.suggest_float('lambda', 3.0, 5.0),
        'alpha': trial.suggest_float('alpha', 1e-5, 9e-4, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3)
    }
    fold_scores = []
    skf_inner = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    for train_idx, val_idx in skf_inner.split(X, grade_for_stratify):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y_true.iloc[train_idx], y_true.iloc[val_idx]
        model = xgb.XGBRegressor(**params, n_estimators=2500, early_stopping_rounds=100)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        preds = model.predict(X_val)
        fold_scores.append(np.sqrt(mean_squared_error(y_val, preds)))
    return np.mean(fold_scores)

N_OPTUNA_TRIALS_KFOLD = 30 # Adjust as needed (30 is a good start)
study_mean = optuna.create_study(direction='minimize')
study_mean.optimize(objective_mean_kfold, n_trials=N_OPTUNA_TRIALS_KFOLD)
best_params_mean = study_mean.best_params
print(f"\n# Mean Model K-Fold Aware Tuning Complete. Best Avg RMSE: ${study_mean.best_value:,.2f}")

# --- STAGE 1, PART 2: Final K-Fold Training of Mean Model ---
print("\n# STAGE 1, PART 2: Final K-Fold Training of Mean Model...")
oof_mean_preds = np.zeros(len(X))
test_mean_preds = np.zeros(len(X_test))
final_params_mean = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'random_state': RANDOM_STATE, 'n_jobs': -1, **best_params_mean}
for fold, (train_idx, val_idx) in enumerate(skf.split(X, grade_for_stratify)):
    print(f"  Mean Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**final_params_mean, n_estimators=2500, early_stopping_rounds=100)
    model.fit(X.iloc[train_idx], y_true.iloc[train_idx], eval_set=[(X.iloc[val_idx], y_true.iloc[val_idx])], verbose=False)
    oof_mean_preds[val_idx] = model.predict(X.iloc[val_idx])
    test_mean_preds += model.predict(X_test) / N_SPLITS

final_mean_rmse = np.sqrt(mean_squared_error(y_true, oof_mean_preds))
print(f"\n# Mean model K-Fold training complete. Final OOF RMSE: ${final_mean_rmse:,.2f}")
print("-" * 50)


--- STAGE 1, PART 1: K-Fold Aware Tuning of Mean Model ---


[I 2025-07-08 22:06:59,556] A new study created in memory with name: no-name-4e11d064-15c5-4f5e-aed2-fd6f7d9d2776
[I 2025-07-08 22:09:55,588] Trial 0 finished with value: 98938.76009298983 and parameters: {'eta': 0.04281475119303402, 'max_depth': 8, 'subsample': 0.8008724938098551, 'colsample_bytree': 0.8282533499349177, 'lambda': 4.582727733350719, 'alpha': 0.00026503269830849513, 'min_child_weight': 3}. Best is trial 0 with value: 98938.76009298983.
[I 2025-07-08 22:12:38,572] Trial 1 finished with value: 99067.04272329391 and parameters: {'eta': 0.04474614584127113, 'max_depth': 8, 'subsample': 0.8698095567011409, 'colsample_bytree': 0.8541165482843458, 'lambda': 4.5231751148301225, 'alpha': 1.6013231735378886e-05, 'min_child_weight': 3}. Best is trial 0 with value: 98938.76009298983.
[I 2025-07-08 22:14:57,408] Trial 2 finished with value: 99148.80405718597 and parameters: {'eta': 0.042785180070228294, 'max_depth': 7, 'subsample': 0.8026216965286678, 'colsample_bytree': 0.820060099


# Mean Model K-Fold Aware Tuning Complete. Best Avg RMSE: $98,770.55

# STAGE 1, PART 2: Final K-Fold Training of Mean Model...
  Mean Model - Fold 1/5...
  Mean Model - Fold 2/5...
  Mean Model - Fold 3/5...
  Mean Model - Fold 4/5...
  Mean Model - Fold 5/5...

# Mean model K-Fold training complete. Final OOF RMSE: $98,772.21
--------------------------------------------------


In [14]:
print("hello world")

hello world


In [15]:
# =============================================================================
# BLOCK 4: K-FOLD AWARE TUNING & TRAINING OF ERROR MODEL
# =============================================================================

# --- Define the Error Target ---
error_target = np.abs(y_true - oof_mean_preds)

# --- Create the Error Model's Feature Set ---
X_for_error = X.copy()
X_for_error['mean_pred_oof'] = oof_mean_preds
X_test_for_error = X_test.copy()
X_test_for_error['mean_pred_oof'] = test_mean_preds

# --- STAGE 2, PART 1: K-Fold Aware Tuning of Error Model ---
print("\n--- STAGE 2, PART 1: K-Fold Aware Tuning of Error Model ---")

def objective_error_kfold(trial):
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method':'hist',
        'random_state': RANDOM_STATE, 'n_jobs': -1,
        # Use the search space from your most successful error model run (v1)
        'eta': trial.suggest_float('eta', 0.01, 0.02),
        'max_depth': trial.suggest_int('max_depth', 8, 10),
        'subsample': trial.suggest_float('subsample', 0.9, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.72, 0.75),
        'lambda': trial.suggest_float('lambda', 0.5, 0.7, log=True),
        'alpha': trial.suggest_float('alpha', 0.5, 0.65),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3)
    }
    fold_scores = []
    skf_inner = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    # Use X_for_error in this split
    for train_idx, val_idx in skf_inner.split(X_for_error, grade_for_stratify):
        X_train, X_val = X_for_error.iloc[train_idx], X_for_error.iloc[val_idx]
        y_train, y_val = error_target.iloc[train_idx], error_target.iloc[val_idx]
        model = xgb.XGBRegressor(**params, n_estimators=2500, early_stopping_rounds=100)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        preds = model.predict(X_val)
        fold_scores.append(np.sqrt(mean_squared_error(y_val, preds)))
    return np.mean(fold_scores)

study_error = optuna.create_study(direction='minimize')
study_error.optimize(objective_error_kfold, n_trials=N_OPTUNA_TRIALS_KFOLD)
best_params_error = study_error.best_params
print(f"\n# Error Model K-Fold Aware Tuning Complete. Best Avg RMSE: ${study_error.best_value:,.2f}")

# --- STAGE 2, PART 2: Final K-Fold Training of Error Model ---
print("\n# STAGE 2, PART 2: Final K-Fold Training of Error Model...")
oof_error_preds = np.zeros(len(X))
test_error_preds = np.zeros(len(X_test))
final_params_error = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'random_state': RANDOM_STATE, 'n_jobs': -1, **best_params_error}
for fold, (train_idx, val_idx) in enumerate(skf.split(X_for_error, grade_for_stratify)):
    print(f"  Error Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**final_params_error, n_estimators=2500, early_stopping_rounds=100)
    model.fit(X_for_error.iloc[train_idx], error_target.iloc[train_idx], eval_set=[(X_for_error.iloc[val_idx], error_target.iloc[val_idx])], verbose=False)
    oof_error_preds[val_idx] = model.predict(X_for_error.iloc[val_idx])
    test_error_preds += model.predict(X_test_for_error) / N_SPLITS

final_error_rmse = np.sqrt(mean_squared_error(error_target, oof_error_preds))
print(f"\n# Error model K-Fold training complete. Final OOF RMSE: ${final_error_rmse:,.2f}")
print("-" * 50)

[I 2025-07-08 23:40:11,084] A new study created in memory with name: no-name-a9e43e31-ad43-40c1-a24b-b7c71fdd43e2



--- STAGE 2, PART 1: K-Fold Aware Tuning of Error Model ---


[I 2025-07-08 23:41:18,646] Trial 0 finished with value: 62651.05659003237 and parameters: {'eta': 0.016004741548190267, 'max_depth': 8, 'subsample': 0.9054208257067495, 'colsample_bytree': 0.7229972362079995, 'lambda': 0.5528965029036256, 'alpha': 0.5679242127910673, 'min_child_weight': 2}. Best is trial 0 with value: 62651.05659003237.
[I 2025-07-08 23:42:35,336] Trial 1 finished with value: 62726.50669290988 and parameters: {'eta': 0.018848903525894523, 'max_depth': 9, 'subsample': 0.942429587879134, 'colsample_bytree': 0.7246729432183672, 'lambda': 0.5027653340251114, 'alpha': 0.5640422618502338, 'min_child_weight': 2}. Best is trial 0 with value: 62651.05659003237.
[I 2025-07-08 23:43:42,711] Trial 2 finished with value: 62743.61984062765 and parameters: {'eta': 0.019318463201290985, 'max_depth': 9, 'subsample': 0.9483914277571206, 'colsample_bytree': 0.7407761878489196, 'lambda': 0.5038826664505884, 'alpha': 0.6346248151809579, 'min_child_weight': 3}. Best is trial 0 with value: 


# Error Model K-Fold Aware Tuning Complete. Best Avg RMSE: $62,621.95

# STAGE 2, PART 2: Final K-Fold Training of Error Model...
  Error Model - Fold 1/5...
  Error Model - Fold 2/5...
  Error Model - Fold 3/5...
  Error Model - Fold 4/5...
  Error Model - Fold 5/5...

# Error model K-Fold training complete. Final OOF RMSE: $62,628.62
--------------------------------------------------


In [17]:
# =============================================================================
# FINAL ASYMMETRIC CALIBRATION AND SUBMISSION (ULTIMATE ROBUST VERSION)
# =============================================================================
print("\n--- Final Asymmetric Calibration ---")

# --- Safely reload y_true to ensure it's available ---
y_true = pd.read_csv('./dataset.csv')['sale_price']
# --- Your existing correct code ---
oof_error_final = np.clip(oof_error_preds, 0, None)
best_a, best_b, best_metric = 2.0, 2.0, float('inf')
for a in np.arange(1.90, 2.31, 0.01):
    for b in np.arange(2.10, 2.51, 0.01):
        low = oof_mean_preds - oof_error_final * a
        high = oof_mean_preds + oof_error_final * b
        # We need the winkler_score function defined here or in a previous cell
        metric, coverage = winkler_score(y_true, low, high, alpha=COMPETITION_ALPHA, return_coverage=True)
        if metric < best_metric:
            best_metric = metric
            best_a, best_b = a, b
print(f"\nGrid search complete. Final OOF Score: {best_metric:,.2f}. Best multipliers: a={best_a:.2f}, b={best_b:.2f}")    

# --- Create Final Submission ---
print("\nCreating final submission file...")
test_error_final = np.clip(test_error_preds, 0, None)
final_lower = test_mean_preds - test_error_final * best_a
final_upper = test_mean_preds + test_error_final * best_b
final_upper = np.maximum(final_lower, final_upper)

# Your excellent, robust fix for the IDs
test_ids = pd.read_csv('./test.csv', usecols=['id'])['id']
submission_df = pd.DataFrame({
    'id': test_ids,
    'pi_lower': final_lower,
    'pi_upper': final_upper
    })

submission_df.to_csv('submission_winner_v1_301.csv', index=False)
print("\n'submission_final_v6.csv' created successfully!")
display(submission_df.head())



--- Final Asymmetric Calibration ---

Grid search complete. Final OOF Score: 302,021.33. Best multipliers: a=1.96, b=2.17

Creating final submission file...

'submission_final_v6.csv' created successfully!


,id,pi_lower,pi_upper
0,200000,838794.670312,1.060866e+06
1,200001,593587.770781,8.215377e+05
2,200002,464649.988477,6.743832e+05
3,200003,281573.330977,4.045716e+05
4,200004,331295.164844,8.423343e+05
